<h1 align = "center">Assignment 3</h1>

# Create a Spark session

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
                    .master('local[1]') \
                    .appName('Assignment') \
                    .getOrCreate()

In [2]:
spark

# Exercise 1

<h3>Given an Input data set with name, age, and city,
if age > 18 add a new column that’s populated with "Y" else "N". </h3>

In [3]:
data1 = spark.read.text("Datasets/exercise 1.txt").rdd

In [4]:
data1.collect()

[Row(value='ram,30,bangalore'),
 Row(value='kapil,32,hyderabad'),
 Row(value='sathish,16,chennai'),
 Row(value='ravi,39,bangalore'),
 Row(value='kavita,12,hyderabad'),
 Row(value='kavya,19,mysore')]

In [5]:
data1Out = data1.flatMap(lambda line:line)\
                .map(lambda line:line.split(','))\
                .map(lambda line : (line,'Y') if int(line[1])>18 else (line,'N'))

In [6]:
data1Out.collect()

[(['ram', '30', 'bangalore'], 'Y'),
 (['kapil', '32', 'hyderabad'], 'Y'),
 (['sathish', '16', 'chennai'], 'N'),
 (['ravi', '39', 'bangalore'], 'Y'),
 (['kavita', '12', 'hyderabad'], 'N'),
 (['kavya', '19', 'mysore'], 'Y')]

# Exercise 2

<h3>Given the input file where columns are stationId, timeOfTheReading, readingType, temperatureRecorded, and few other columns.We need to find the minimum temperature for each stationId.</h3>


In [7]:
data2 = spark.read.text("Datasets/exercise 2.txt").rdd

In [8]:
data2.collect()

[Row(value='ITE00100554,18000101,TMAX,-75,,,E,'),
 Row(value='ITE00100554,18000101,TMIN,-148,,,E,'),
 Row(value='GM000010962,18000101,PRCP,0,,,E,'),
 Row(value='EZE00100082,18000101,TMAX,-86,,,E,'),
 Row(value='EZE00100082,18000101,TMIN,-135,,,E,'),
 Row(value='ITE00100554,18000102,TMAX,-60,,I,E,'),
 Row(value='ITE00100554,18000102,TMIN,-125,,,E,'),
 Row(value='GM000010962,18000102,PRCP,0,,,E,'),
 Row(value='EZE00100082,18000102,TMAX,-44,,,E,'),
 Row(value='EZE00100082,18000102,TMIN,-130,,,E,'),
 Row(value='ITE00100554,18000103,TMAX,-23,,,E,'),
 Row(value='ITE00100554,18000103,TMIN,-46,,I,E,'),
 Row(value='GM000010962,18000103,PRCP,4,,,E,'),
 Row(value='EZE00100082,18000103,TMAX,-10,,,E,'),
 Row(value='EZE00100082,18000103,TMIN,-73,,,E,'),
 Row(value='ITE00100554,18000104,TMAX,0,,,E,'),
 Row(value='ITE00100554,18000104,TMIN,-13,,,E,'),
 Row(value='GM000010962,18000104,PRCP,0,,,E,'),
 Row(value='EZE00100082,18000104,TMAX,-55,,,E,'),
 Row(value='EZE00100082,18000104,TMIN,-74,,,E,'),
 Row

In [9]:
def minimum(idTuple):
    values = idTuple[1]
    min_ = 0
    flag = False
    for i in values:
        if flag:
            min_ = int(i[3])
            flag = True
        elif min_ > int(i[3]):
            min_ = int(i[3])
    return idTuple[0],min_

In [10]:
data2Out = data2.flatMap(lambda line:line)\
                .map(lambda line:line.split(','))\
                .groupBy(lambda columns : columns[0])\
                .map(lambda idTuple: minimum(idTuple))

In [11]:
data2Out.collect()

[('ITE00100554', -148), ('GM000010962', 0), ('EZE00100082', -135)]

# Exercise 3

<h3>Given the input data, find the top 10 words searched.</h3>

In [12]:
data3 = spark.read.text("Datasets/exercise 3.txt").rdd

In [13]:
data3.collect()

[Row(value='best big data hadoop spark training'),
 Row(value='kelly technologies courses'),
 Row(value='intellepaat'),
 Row(value='cloudera training partner india'),
 Row(value='microsoft certification courses in big data'),
 Row(value='big data online preparation'),
 Row(value='BIG DATA COURSE IN DELHI'),
 Row(value='online data mining certification programs'),
 Row(value='acadgild'),
 Row(value='BIG DATA coursera module 1'),
 Row(value='PYTHON training in chennai'),
 Row(value='hadoop developer'),
 Row(value='microsoft big data certification'),
 Row(value='hadoop training in delhi'),
 Row(value='big data courses in hyderabad online'),
 Row(value='computer data course'),
 Row(value='big data HADOOP training report'),
 Row(value='DATA analytics course in bangalore'),
 Row(value='cloudera certified professional'),
 Row(value='apache hadoop institute in delhi'),
 Row(value='good apache spark classes'),
 Row(value='spark interview questions'),
 Row(value='big data contents'),
 Row(value=

In [14]:
data3Out = data3.flatMap(lambda line:line)\
                .flatMap(lambda line:line.split(' '))\
                .map(lambda word: (word.lower(),1))\
                .reduceByKey(lambda a,b: a+b)\
                .sortBy(lambda word:word[1],ascending=False)

In [15]:
data3Out.take(10)

[('data', 361),
 ('big', 285),
 ('in', 171),
 ('training', 114),
 ('course', 105),
 ('hadoop', 100),
 ('online', 58),
 ('courses', 53),
 ('spark', 42),
 ('bangalore', 40)]

# Exercise 4

<h3>Find the top 10 customers who spend the maximum amount on shopping.</h3>
<h4>File Details :CustomerId, ProductId, Amount</h4>

In [16]:
data4 = spark.read.text("Datasets/exercise 4.txt").rdd

In [17]:
data4.collect()

[Row(value='44,8602,37.19'),
 Row(value='35,5368,65.89'),
 Row(value='2,3391,40.64'),
 Row(value='47,6694,14.98'),
 Row(value='29,680,13.08'),
 Row(value='91,8900,24.59'),
 Row(value='70,3959,68.68'),
 Row(value='85,1733,28.53'),
 Row(value='53,9900,83.55'),
 Row(value='14,1505,4.32'),
 Row(value='51,3378,19.80'),
 Row(value='42,6926,57.77'),
 Row(value='2,4424,55.77'),
 Row(value='79,9291,33.17'),
 Row(value='50,3901,23.57'),
 Row(value='20,6633,6.49'),
 Row(value='15,6148,65.53'),
 Row(value='44,8331,99.19'),
 Row(value='5,3505,64.18'),
 Row(value='48,5539,32.42'),
 Row(value='47,9900,25.66'),
 Row(value='31,7570,4.16'),
 Row(value='4,7577,24.13'),
 Row(value='36,8274,88.64'),
 Row(value='57,963,57.91'),
 Row(value='12,4396,72.62'),
 Row(value='22,7161,56.06'),
 Row(value='54,7765,28.01'),
 Row(value='0,3479,97.22'),
 Row(value='88,1272,80.70'),
 Row(value='86,9254,71.90'),
 Row(value='13,4507,26.47'),
 Row(value='88,1335,29.76'),
 Row(value='42,6601,22.98'),
 Row(value='40,3083,72.9

In [18]:
def mapper4(line):
    tokens = line.split(',')
    return tokens[0],float(tokens[2])

data4Out = data4.flatMap(lambda line:line)\
                .map(lambda line:mapper4(line))\
                .reduceByKey(lambda a,b :a+b)\
                .sortBy(lambda line:line[1],ascending=False)

In [19]:
data4Out.take(10)

[('68', 6375.449999999997),
 ('73', 6206.199999999999),
 ('39', 6193.109999999999),
 ('54', 6065.389999999999),
 ('71', 5995.660000000003),
 ('2', 5994.59),
 ('97', 5977.189999999995),
 ('46', 5963.109999999999),
 ('42', 5696.840000000003),
 ('59', 5642.89)]

# Exercise 5

<h3>Given a data set, write a spark code to answer below questions.
<ul> 
    <li>How many times do movies rated 5 stars?</li>
    <li>How many times do movies rated 4 stars?</li>
    <li>How many times do movies rated 3 stars?</li>
    <li>How many times do movies rated 2 start?</li>
    <li>How many times do movies rated 1 start?</li>
</ul>
Data Set Columns Respectively: userid, movieid, rating, timestamp</h3>

In [20]:
data5 = spark.read.text("Datasets/exercise 5.txt").rdd

In [21]:
data5.collect()

[Row(value='196\t242\t3\t881250949'),
 Row(value='186\t302\t3\t891717742'),
 Row(value='22\t377\t1\t878887116'),
 Row(value='244\t51\t2\t880606923'),
 Row(value='166\t346\t1\t886397596'),
 Row(value='298\t474\t4\t884182806'),
 Row(value='115\t265\t2\t881171488'),
 Row(value='253\t465\t5\t891628467'),
 Row(value='305\t451\t3\t886324817'),
 Row(value='6\t86\t3\t883603013'),
 Row(value='62\t257\t2\t879372434'),
 Row(value='286\t1014\t5\t879781125'),
 Row(value='200\t222\t5\t876042340'),
 Row(value='210\t40\t3\t891035994'),
 Row(value='224\t29\t3\t888104457'),
 Row(value='303\t785\t3\t879485318'),
 Row(value='122\t387\t5\t879270459'),
 Row(value='194\t274\t2\t879539794'),
 Row(value='291\t1042\t4\t874834944'),
 Row(value='234\t1184\t2\t892079237'),
 Row(value='119\t392\t4\t886176814'),
 Row(value='167\t486\t4\t892738452'),
 Row(value='299\t144\t4\t877881320'),
 Row(value='291\t118\t2\t874833878'),
 Row(value='308\t1\t4\t887736532'),
 Row(value='95\t546\t2\t879196566'),
 Row(value='38\t95\t

In [22]:
def mapper5(line):
    tokens = line.split('\t')
    return tokens[2]+" Star Movies",1

data5Out = data5.flatMap(lambda line:line)\
                .map(lambda line:mapper5(line))\
                .reduceByKey(lambda a,b :a+b)\
                .sortByKey(False)

In [23]:
data5Out.collect()

[('5 Star Movies', 21201),
 ('4 Star Movies', 34174),
 ('3 Star Movies', 27145),
 ('2 Star Movies', 11370),
 ('1 Star Movies', 6110)]